In [65]:
import os
import pandas as pd
import numpy as np
import requests
from pprint import pprint
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from config import password

In [66]:
# Get api data
url = "https://api.covidtracking.com/v1/states/current.json"
response = requests.get(url).json()

In [67]:
# Get data for  dataframe
# Determine the length of the response so we can use that for the number of states/territories
len(response)
num_state = range(0,56)
# Create empty lists
state = []
date = []
death = []
# Honestly, was super proud of this part bc i am BAD AT FOR LOOPS but Google/Zed Shaw is my religion now
# Create for loop to get the data that we need  for state, date, and death
for i in num_state:
    (print(response[i]["state"]))
    state.append(response[i]["state"])
    (print(response[i]["date"]))
    date.append(response[i]["date"])
    (print(response[i]["deathConfirmed"]))
    death.append(response[i]["death"])

AK
20201027
70
AL
20201027
2699
AR
20201027
1696
AS
20201027
None
AZ
20201027
5601
CA
20201027
None
CO
20201027
None
CT
20201027
3687
DC
20201027
None
DE
20201027
605
FL
20201027
16709
GA
20201027
None
GU
20201027
None
HI
20201027
None
IA
20201027
None
ID
20201027
536
IL
20201027
9568
IN
20201027
3958
KS
20201027
None
KY
20201027
1408
LA
20201027
5666
MA
20201027
9664
MD
20201027
3962
ME
20201027
145
MI
20201027
7239
MN
20201027
2354
MO
20201027
None
MP
20201027
2
MS
20201027
2964
MT
20201027
None
NC
20201027
4144
ND
20201027
361
NE
20201027
None
NH
20201027
None
NJ
20201027
14517
NM
20201027
None
NV
20201027
None
NY
20201027
None
OH
20201027
4927
OK
20201027
None
OR
20201027
None
PA
20201027
None
PR
20201027
616
RI
20201027
None
SC
20201027
3602
SD
20201027
None
TN
20201027
3037
TX
20201027
None
UT
20201027
None
VA
20201027
3350
VI
20201027
21
VT
20201027
58
WA
20201027
2321
WI
20201027
1852
WV
20201027
428
WY
20201027
None


In [68]:
# Create covid dataframe ffrom api data
covid_deaths = pd.DataFrame({"state":state, "date": date, "death":death})
covid_df.head()

,state,date,death
0,AK,20201026,68
1,AL,20201026,2866
2,AR,20201026,1833
3,AS,20201026,0
4,AZ,20201026,5875


In [69]:
# Import csv that includes data for Total Cases by state
# Filepath
csv_filepath = os.path.join("data","cdc_covid.csv")
# Read csv into DataFrame
covid_cases = pd.read_csv(csv_filepath)
# Review head and shape
covid_cases.head()
covid_cases.shape

(61, 13)

In [70]:
# Drop unnecessary columns from covid_cases
covid_cases = covid_cases.drop(columns = ["Confirmed Cases", "Total Deaths", "Probable Cases", "Cases in Last 7 Days", "Case Rate per 100000", "Confirmed Deaths", "Probable Deaths", "Deaths in Last 7 Days", "Death Rate per 100000", "Case Rate per 100000 in Last 7 Days", "Death Rate per 100K in Last 7 Days"])
# Review head
covid_cases.head()

,State/Territory,Total Cases
0,Alaska,12118
1,Alabama,180916
2,Arkansas,104135
3,American Samoa,0
4,Arizona,235882


In [71]:
# Read in table to convert state name to state code
# Filepaths
state_csv = os.path.join("data","csvData.csv")
# Read csv into DataFrame
state_table = pd.read_csv(state_csv)

In [72]:
state_table.head()

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [73]:
covid_deaths_state = pd.merge(covid_deaths, state_table, left_on="state", right_on="Code", how="inner")

In [74]:
covid_deaths_state.head()

,state,date,death,State,Abbrev,Code
0,AK,20201027,70,Alaska,Alaska,AK
1,AL,20201027,2892,Alabama,Ala.,AL
2,AR,20201027,1857,Arkansas,Ark.,AR
3,AZ,20201027,5891,Arizona,Ariz.,AZ
4,CA,20201027,17400,California,Calif.,CA


In [75]:
final_data = pd.merge(covid_df_state, covid_cases, left_on="State", right_on="State/Territory", how="inner")
final_data.head()

,state,date,death,State,Abbrev,Code,State/Territory,Total Cases
0,AK,20201026,68,Alaska,Alaska,AK,Alaska,12118
1,AL,20201026,2866,Alabama,Ala.,AL,Alabama,180916
2,AR,20201026,1833,Arkansas,Ark.,AR,Arkansas,104135
3,AZ,20201026,5875,Arizona,Ariz.,AZ,Arizona,235882
4,CA,20201026,17357,California,Calif.,CA,California,886865


In [76]:
# Limit the columns of the final data
final_data = final_data[["State/Territory", "date", "Code", "death", "Total Cases"]]
final_data.head()

,State/Territory,date,Code,death,Total Cases
0,Alaska,20201026,AK,68,12118
1,Alabama,20201026,AL,2866,180916
2,Arkansas,20201026,AR,1833,104135
3,Arizona,20201026,AZ,5875,235882
4,California,20201026,CA,17357,886865


In [77]:
# Rename columns in preparation for Postgres connection
final_data = final_data.rename(columns = {"State/Territory": "state", "Total Cases": "total_cases", "Code": "code"})
final_data.head()

,state,date,code,death,total_cases
0,Alaska,20201026,AK,68,12118
1,Alabama,20201026,AL,2866,180916
2,Arkansas,20201026,AR,1833,104135
3,Arizona,20201026,AZ,5875,235882
4,California,20201026,CA,17357,886865


In [78]:
# Convert date column to actual date
final_data['date'] = pd.to_datetime(final_data['date'].astype(str), format='%Y%m%d')
final_data.head()

,state,date,code,death,total_cases
0,Alaska,2020-10-26,AK,68,12118
1,Alabama,2020-10-26,AL,2866,180916
2,Arkansas,2020-10-26,AR,1833,104135
3,Arizona,2020-10-26,AZ,5875,235882
4,California,2020-10-26,CA,17357,886865


In [83]:
# Connect to database
pg_user = "postgres"
pg_password = password
db_name = "covid_ETL"
connection_string = f"{pg_user}:{password}@localhost:5432/{db_name}"
engine = create_engine(f"postgresql://{connection_string}")

In [84]:
engine.table_names()

[]

In [85]:
final_data.to_sql(name="covid_table", con=engine, if_exists="append", index=False)